In [1]:
import pygame
from pygame.locals import *
from pygame import gfxdraw
import sys
import numpy as np
import copy

import tensorflow as tf

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver

from scan_gym import envs

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
BROWN = (139,69,19)
BLACK = (0,0,0)
RED = (255,0,0)
GREEN = (0,255,0)

def get_circle_points(center,radius,npoints):
    angleStep = (2*np.pi)/npoints
    angles = np.arange(0,2*np.pi,angleStep)
    
    points = []
    #get points
    for i in angles:
        point = np.array([int(radius*np.cos(i)),int(radius*np.sin(i))])
        #translate point according to circle center
        point += center
        points.append(point)    
    return points


def convert_images(images):
    imgs=[]
    for i in images:
        mode = i.mode
        size = i.size
        data = i.tobytes()
        py_image = pygame.image.fromstring(data, size, mode)
        py_image = pygame.transform.scale(py_image, (640, 358))
        #img = pygame.transform.flip(img, False, True)
        imgs.append(py_image)
    return imgs


class policySim():
    def __init__(self,imgs):
        self.imgs = imgs
        pygame.display.set_caption('cnc simulator')
        size = 1024
        self.width = size
        self.height = size
        self.background = pygame.Surface((self.width,self.height))
        self.background.fill(BROWN)
        self.npoints = 720
        self.radius = int((size*.9)/2.0)
        self.cam_pos = get_circle_points((int(self.width/2),int(self.height/2)),self.radius,self.npoints)
        pygame.draw.circle(self.background, BLACK ,(int(self.width/2),int(self.height/2)), self.radius,4)

        self.screen = pygame.display.set_mode((self.width, self.height))
        self.screen.blit(self.background, (0, 0))
        pygame.display.update()

    def reset(self):
        self.background = pygame.Surface((self.width,self.height))
        self.background.fill(BROWN)
        pygame.draw.circle(self.background, BLACK ,(int(self.width/2),int(self.height/2)), self.radius,4)
        self.screen.blit(self.background, (0, 0))
        pygame.display.update()

    def move(self,position):
        self.screen.blit(self.background, (0, 0))
        self.screen.blit(self.imgs[position], (192, 323))
        pygame.draw.circle(self.screen, RED , self.cam_pos[position], 15)
        pygame.display.update()

    def save(self,position):
        #gfxdraw.pixel(self.background, self.cam_pos[position][0], self.cam_pos[position][1], GREEN)
        pygame.draw.circle(self.background, GREEN , self.cam_pos[position], 1)
        pygame.draw.circle(self.screen, GREEN , self.cam_pos[position], 15)
        pygame.display.update()


In [3]:
policy_dir = '/home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/multienv_policy_100000'  #os.path.join(current_path, 'policy')
policy = tf.compat.v2.saved_model.load(policy_dir)

In [4]:
dp = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/000'
env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp})
test_env = tf_py_environment.TFPyEnvironment( env )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
pygame.init()
imgs=convert_images(env.rwd_images)
sim = policySim(imgs)
delay = 50

In [6]:
state = test_env.reset()
#print(state.observation[0].numpy())
last_position = env.current_position
cont = 0
time_steps = 3000
#obs = deque(maxlen=time_steps)

for i in range(time_steps):
    #obs.append(state.observation[0].numpy())
    cont +=1
    action_step = policy.action(state)#agent.policy.action(state)
    next_state = test_env.step(action_step.action)
    
    sim.move(env.current_position)
    action = env.action2move(action_step.action.numpy(),state.observation[1].numpy()[0][-1])
    if action == 88888:
        sim.save(env.current_position)
    pygame.time.delay(delay)

    print(cont,':',last_position,state.observation[1].numpy(),action ) #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    state = copy.deepcopy(next_state)
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_images)
print(sorted(env.kept_images))
print(env.total_reward)
print(len(env.kept_images))
print(env.total_moves)

NotFoundError:  No algorithm worked!
	 [[{{node StatefulPartitionedCall/QNetwork/EncodingNetwork/model/conv2d/Relu}}]] [Op:__inference_restored_function_body_1186]

Function call stack:
restored_function_body


In [ ]:
env.chamfer_from_collected()